In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import the sbg workspace and local utils
import sys
sys.path.append('/home/leandro/Dropbox/workspacesbg/sbg/')
sys.path.append('../../../code/')

from sbg.orf.Orf import OnlineOrf
from sbg.structure.AlphaFoldHandler import AlphaFoldHandler
from sbg.structure.Structure import Structure
from sbg.common.FileHandler import FileHandler
from sbg.scripts.foldx.TangoHandler import TangoHandler

import pipeline_functions as pf

In [3]:
# External libraries
import pandas as pd
import os
from sequence_utils import highlight_sequence, align_sequences_biopython, display_alignment_with_highlighting


In [4]:
# Load the data from data/protein_data.csv
data = pd.read_csv('data/protein_data.csv', index_col=0)

# Computation Pipeline

In [5]:
entry = 0
for up_acc in data.index:
    try:
        entry += 1
        print(f'Processing {up_acc} {data.loc[up_acc, 'swiss_prot']}, {entry} of {len(data.index)}')

        results_dir = f'/home/leandro/Insync/gdrive/Mimark/code/cohort-analysis/notebooks/other_analyses/antigens/data/results/{up_acc[0:2]}/{up_acc}/'
        FileHandler.ensureDir(results_dir)

        # Step 1: Get the Uniprot object
        step = 1
        uniprot_object = OnlineOrf(up_acc)
        step = 1.1
        glycosilation_sites = uniprot_object.getGlycosylationSites()
        if len(uniprot_object.getGlycosylationSites()) > 0:
            glycosylation_df = pd.DataFrame(uniprot_object.getGlycosylationSites(), columns=['res', 'glycosylation'])
            glycosylation_df.to_csv(results_dir + f'{up_acc}_glycosylation.csv')
        
        step = 1.2
        modified_residues = uniprot_object.getModifiedResidues()
        if len(uniprot_object.getModifiedResidues()) > 0:
            modified_residues_df = pd.DataFrame(uniprot_object.getModifiedResidues(), columns=['res', 'modification'])
            modified_residues_df.to_csv(results_dir + f'{up_acc}_modifications.csv')
        
        step = 1.3
        try:
            isoforms = uniprot_object.getIsoforms()
        except:
            isoforms = []
        step = 1.4
        uniprot_string_id = uniprot_object.getStringId()
        step = 1.5
        try:
            subcellular_location = uniprot_object.getSubcellularLocation()
        except:
            subcellular_location = None
        step=1.6
        crytals = uniprot_object.getCrystals()

        # Step 2: Get the AlphaFold object
        step = 2
        alphafold_object = AlphaFoldHandler().get_model(up_acc)

        # Step 3: Run discotope 3.0 and save the results, then load them
        step = 3
        if alphafold_object is not None:
            discotope3_file = f'{up_acc}_A_discotope3.csv'
            # If no results, run discotope
            # if not FileHandler.fileExists(path=results_dir + discotope3_file):
            #     pf.run_discotope(up_acc, save_to=results_dir)
            
            # Now, if the results are there, load them
            if FileHandler.fileExists(path=results_dir + discotope3_file):
                discotope_results = pd.read_csv(results_dir + discotope3_file, index_col=0)
            else:
                discotope_results = None

        # Step 4: Compute the expression 
        step = 4
        exp_file = f'{up_acc}_exp_results.csv'
        if FileHandler.fileExists(results_dir + exp_file):
            exp_results = pd.read_csv(results_dir + exp_file, index_col=0)
        else:
            try:
                exp_auc, exp_normal, exp_cancer = pf.compare_gene_expression(gene_name=uniprot_object.getGeneName(), up_acc=up_acc, save_to=results_dir)
            except:
                exp_auc, exp_normal, exp_cancer = None, None, None
            
            exp_results = pd.DataFrame({'exp_auc': [exp_auc], 'exp_normal': [exp_normal], 'exp_cancer': [exp_cancer]})
            exp_results.to_csv(results_dir + exp_file)

        # Step 5: Align the isoforms
        step = 5
        if not FileHandler.fileExists(results_dir + f'{up_acc}_isoforms_alignment.html') and len(isoforms) > 1:
            secuences = []
            names = []
            for isoform in isoforms:
                isoform_obj = OnlineOrf(isoform)
                secuences.append(isoform_obj.getSequence())
                names.append(isoform)
            alignment = align_sequences_biopython(secuences, names)
            display_alignment_with_highlighting(alignment, save_to=results_dir + f'{up_acc}_isoforms_alignment.html')

        # Step 6: Compute the aggregation propensity
        step = 6
        if alphafold_object is not None:
            if not FileHandler.fileExists(results_dir + f'{up_acc}_agg.txt'):
                TangoHandler.getAggregation(alphafold_object,results_dir)

            agg_df = pd.read_csv(results_dir + f'{up_acc}_agg.txt', sep='\t', header=0, index_col="res")[["Aggregation"]]
        
        # Step 7: Get interaction partners
        step = 7
        if not FileHandler.fileExists(results_dir + f'{up_acc}_interactors.tsv'):
            interactors = pf.get_interactors(uniprot_string_id, up_acc, save_to=results_dir)
        else:
            interactors = pd.read_csv(results_dir + f'{up_acc}_interactors.tsv', sep='\t', index_col=0)

        # Step 8: Get the protein homology
        step = 8
        if not FileHandler.fileExists(results_dir + f'{up_acc}_homologs.tsv'):
            homology = pf.get_homologs(uniprot_object.getGeneName(), up_acc, save_to=results_dir)
        else:
            homology = pd.read_csv(results_dir + f'{up_acc}_homologs.tsv', sep='\t', index_col=0)

        # Step 9: Get the protein structures bioassemblies to check for homo and multimers
        step = 9
        if not FileHandler.fileExists(results_dir + f'{up_acc}_bioassemblies.csv'):
            try:
                all_dataframes = []
                for crystal in crytals:
                    structure = Structure(crystal, replaceExistent=False)
                    df = structure.analyzeStructureType()
                    df['crystal_id'] = crystal
                    all_dataframes.append(df)

                if len(all_dataframes) > 1:
                    bioassemblies_df = pd.concat(all_dataframes)
                elif len(all_dataframes) == 1:
                    bioassemblies_df = all_dataframes[0]
                else:
                    bioassemblies_df = None

                if len(all_dataframes) > 0:
                    bioassemblies_df.to_csv(results_dir+f'{up_acc}_bioassemblies.csv')
            except:
                bioassemblies_df = None
        else:
            bioassemblies_df = pd.read_csv(results_dir + f'{up_acc}_bioassemblies.csv')

    except Exception as e:
        print(f'Error with {up_acc} in step {step}: {e}')

Processing P07355 ANXA2_HUMAN, 1 of 505
0.78 13.151248717948716 13.968305263157895
Processing O00425 IF2B3_HUMAN, 2 of 505
0.99 1.1419833333333336 9.065943859649124
Processing O15392 BIRC5_HUMAN, 3 of 505
1.0 3.6224346153846145 11.729612280701755
Processing P00749 UROK_HUMAN, 4 of 505
0.71 9.030235897435897 9.84464912280702
Processing P02787 TRFE_HUMAN, 5 of 505
0.37 6.784505128205128 6.344794736842104
Processing P03372 ESR1_HUMAN, 6 of 505
0.0 13.055671794871795 7.592115789473684
Processing P04792 HSPB1_HUMAN, 7 of 505
0.26 15.26426282051282 14.451017543859649
Processing P07339 CATD_HUMAN, 8 of 505
0.89 13.183726923076923 14.209715789473682
Processing P08253 MMP2_HUMAN, 9 of 505
0.28 14.297517948717951 13.429126315789473
Processing P09603 CSF1_HUMAN, 10 of 505
0.28 11.161249999999999 10.31678947368421
Processing P10415 BCL2_HUMAN, 11 of 505
0.06 11.15190512820513 9.09681403508772
Processing P11166 GTR1_HUMAN, 12 of 505
0.95 9.952429487179485 12.47030701754386
Processing P14780 MMP9_HU